In [2]:
from collections import Counter
from itertools import combinations, chain

import networkx
from networkx import connected_components
import numpy as np
import pandas as pd

In [8]:
pd.set_option('display.max_columns', None)

In [162]:
ens_annot = pd.read_csv('../data/processed/ensembl_annotation_20220705.csv', 
                        dtype={'chromosome_or_scaffold': 'O', 
                               '5utr_start': 'Int64', 
                               '5utr_end': 'Int64', 
                               '3utr_start': 'Int64', 
                               '3utr_end': 'Int64', 
                               'cds_start': 'Int64', 
                               'cds_end': 'Int64'})

ens_exons = pd.read_csv('../data/processed/ensembl_exons_20220705.csv', 
                        dtype={'5utr_start': 'Int64', 
                               '5utr_end': 'Int64', 
                               '3utr_start': 'Int64', 
                               '3utr_end': 'Int64', 
                               'cds_start': 'Int64', 
                               'cds_end': 'Int64'})

In [163]:
ens_exons = ens_exons.set_index(['ensembl_trs_id', 'ensembl_exon_id'])
ens_annot = ens_annot.set_index('ensembl_trs_id')

In [141]:
def connect_components(edge_list):
    G = networkx.Graph()
    G.add_edges_from(edge_list)
    
    return [list(i) for i in connected_components(G)]

In [147]:
def extract_duplicate_cds_trs(gene):
    trs_pc_cds_arrs = []
    
    trs_pc = ens_annot[(ens_annot['ensembl_gene_id'] == gene) & (ens_annot['trs_type'] == 'protein_coding')].index.values
    
    for trs in trs_pc:
        cds_arr = np.sort(ens_exons.loc[trs, ['cds_start', 'cds_end']].dropna().to_numpy().ravel())
        trs_pc_cds_arrs.append(cds_arr)
    
    results = []
    
    # Pairwise check of equality between cds arrays = transcripts
    for pair in combinations(range(len(trs_pc_cds_arrs)), 2):
        if np.array_equal(trs_pc_cds_arrs[pair[0]], trs_pc_cds_arrs[pair[1]]):
            results.append(list(trs_pc[list(pair)]))
    
    # Connect pairs of trs with equal cds if possible
    if results:
        results = connect_components(results)
        return results
    else:
        return np.nan

In [164]:
ens_annot = ens_annot[ens_annot['chromosome_or_scaffold'].isin([str(i) for i in list(range(1,23))] + ['X', 'Y'])]
genes_pc = ens_annot[ens_annot['trs_type'] == 'protein_coding'].groupby('ensembl_gene_id').size()
genes_pc = genes_pc[genes_pc >= 2].index

In [145]:
len(genes_pc)

15211

In [148]:
genes_dup = pd.Series(index=genes_pc, dtype='O')

for i, gene in enumerate(genes_pc):
    genes_dup[gene] = extract_duplicate_cds_trs(gene)
    print(f'Done {i+1} out of {len(genes_dup)}        ', end='\r')

In [151]:
genes_dup = genes_dup[~genes_dup.isna()]

In [165]:
ens_annot['_protein_id'] = ens_annot['uniprot_isoform_id'].combine_first(ens_annot['uniprot_trembl_id']).combine_first(ens_annot['ensembl_protein_id'])

In [153]:
dup_all = genes_dup.explode().explode().values

In [166]:
assert ~ens_annot[ens_annot.index.isin(dup_all)]['_protein_id'].isna().any()

In [155]:
genes_dup = genes_dup.explode()

In [156]:
conflicts = []
uniprot_id_conflicts = []

for i, j in enumerate(genes_dup.values):
    prot_ids = ens_annot.loc[j, '_protein_id'].to_numpy()
    
    if not (prot_ids == prot_ids[0]).all():
        conflicts.append(i)
        prot_ids_ranked = ens_annot.loc[j, ['uniprot_isoform_id', 'uniprot_trembl_id', 'ensembl_protein_id']].melt().dropna()
        if 'uniprot_isoform_id' in prot_ids_ranked['variable'].to_list():
            uniprot_id_conflicts.append(i)
        ens_annot.loc[j, '_protein_id'] = prot_ids_ranked['value'].iloc[0]
        
    print(f'{i+1} / {len(genes_dup.values)}        ', end='\r')

In [168]:
len(conflicts)

871

In [169]:
len(uniprot_id_conflicts)

623

In [159]:
genes_dup.apply(lambda x: len(x)).sort_values(ascending=False)

ensembl_gene_id
ENSG00000164733    39
ENSG00000121940    33
ENSG00000169255    31
ENSG00000157601    30
ENSG00000087053    29
                   ..
ENSG00000134313     2
ENSG00000134308     2
ENSG00000134291     2
ENSG00000134287     2
ENSG00000145725     2
Length: 7393, dtype: int64

In [175]:
ens_annot['_protein_id'] = ens_annot['_protein_id'].combine_first(ens_annot['uniprot_isoform_id']).combine_first(ens_annot['uniprot_trembl_id']).combine_first(ens_annot['ensembl_protein_id'])

In [181]:
assert (~ens_annot.loc[ens_annot['trs_type'] == 'protein_coding', '_protein_id'].isna()).all()

In [182]:
ens_annot.to_csv('../data/interim/ensembl_annotation_with_dups_20220705.csv')